<a href="https://colab.research.google.com/github/thetsuthetsu/PythonSamples/blob/main/pytorch%E3%83%9F%E3%83%8B%E3%83%90%E3%83%83%E3%83%81%E5%AD%A6%E7%BF%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ディープラーニング入門４　https://www.youtube.com/watch?v=TrRHMjhlq_w&t=6557s
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
from sklearn.datasets import load_iris

In [4]:
iris = load_iris()

In [5]:
# 入力値と目標値を抽出
x = iris['data']
t = iris['target']

In [6]:
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [7]:
type(x), type(t)

(numpy.ndarray, numpy.ndarray)

In [8]:
# PytorchのTensor型へ変換
x = torch.tensor(x, dtype=torch.float32)

In [11]:
t = torch.tensor(t, dtype=torch.int64)

<ipython-input-11-ea86866884e9>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t = torch.tensor(t, dtype=torch.int64)


In [12]:
type(x), type(t)

(torch.Tensor, torch.Tensor)

In [13]:
x.shape, t.shape

(torch.Size([150, 4]), torch.Size([150]))

In [13]:
# x.shape -> 150サンプル, 4入力値 (がく片の長さ、がく片の幅、花びらの長さ、花びらの幅)

In [14]:
# t.shape -> 150サンプル（分類値)

In [14]:

## DataLoaderの定義：Miniバッチ処理に必要
# 入力値と目標値をまとめる
dataset = torch.utils.data.TensorDataset(x,t)

In [15]:
len(dataset)

150

In [16]:
dataset[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor(0))

In [18]:
# データセット分割
# 学習データ(60)、検証データ(20)、テストデータ(20) ^

In [17]:
# 各データのサンプル数を決定
# train : val : test = 60 : 20 : 20
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val

In [18]:
n_train, n_val, n_test

(90, 30, 30)

In [19]:
torch.manual_seed(0)

In [20]:
# データセットの分割
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

In [21]:
len(train), len(val), len(test)

(90, 30, 30)

In [22]:
## ミニバッチ学習
# バッチサイズ (目安：学習サンプル数の1/10～1/100)
batch_size = 10

In [23]:
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)

In [24]:
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

In [25]:
# １つ目のバッチ結果を取り出す
x,t = next(iter(train_loader))

In [26]:
x

tensor([[7.7000, 3.8000, 6.7000, 2.2000],
        [5.0000, 3.4000, 1.6000, 0.4000],
        [5.5000, 3.5000, 1.3000, 0.2000],
        [6.4000, 2.8000, 5.6000, 2.2000],
        [6.7000, 2.5000, 5.8000, 1.8000],
        [7.7000, 3.0000, 6.1000, 2.3000],
        [5.1000, 2.5000, 3.0000, 1.1000],
        [5.0000, 3.6000, 1.4000, 0.2000],
        [7.3000, 2.9000, 6.3000, 1.8000],
        [6.1000, 2.8000, 4.0000, 1.3000]])

In [27]:
t

tensor([2, 0, 0, 2, 2, 2, 1, 0, 2, 1])

In [31]:
## ネットワークの定義 (4 -> 4 -> 3の全結合層)
# fc1: input:4 => output:4
# fc2: input:4 => output:3


In [28]:
class Net(nn.Module):
  # 使用するオブジェクトを定義
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(4,4)
    self.fc2 = nn.Linear(4,3)

  # 順伝搬
  def forward(self, x):
    h = self.fc1(x)
    h = F.relu(h)
    h = self.fc2(h)
    return h


In [29]:
torch.manual_seed(0)
net = Net()

In [30]:
net

Net(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
)

In [31]:
# 最適化手法の定義 (SGD, 学習係数:0.01)
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

In [32]:
# １バッチ分の学習
batch = next(iter(train_loader))

In [33]:
batch

[tensor([[5.4000, 3.9000, 1.7000, 0.4000],
         [4.6000, 3.6000, 1.0000, 0.2000],
         [6.5000, 3.0000, 5.5000, 1.8000],
         [6.9000, 3.1000, 5.4000, 2.1000],
         [6.3000, 2.5000, 4.9000, 1.5000],
         [7.1000, 3.0000, 5.9000, 2.1000],
         [5.8000, 2.7000, 4.1000, 1.0000],
         [7.0000, 3.2000, 4.7000, 1.4000],
         [6.7000, 3.0000, 5.0000, 1.7000],
         [7.2000, 3.6000, 6.1000, 2.5000]]),
 tensor([0, 0, 2, 2, 1, 2, 1, 1, 1, 2])]

In [34]:
x,t = batch

In [35]:
x

tensor([[5.4000, 3.9000, 1.7000, 0.4000],
        [4.6000, 3.6000, 1.0000, 0.2000],
        [6.5000, 3.0000, 5.5000, 1.8000],
        [6.9000, 3.1000, 5.4000, 2.1000],
        [6.3000, 2.5000, 4.9000, 1.5000],
        [7.1000, 3.0000, 5.9000, 2.1000],
        [5.8000, 2.7000, 4.1000, 1.0000],
        [7.0000, 3.2000, 4.7000, 1.4000],
        [6.7000, 3.0000, 5.0000, 1.7000],
        [7.2000, 3.6000, 6.1000, 2.5000]])

In [36]:
t

tensor([0, 0, 2, 2, 1, 2, 1, 1, 1, 2])

In [37]:
# 予測値の算出
y = net.forward(x)
y

tensor([[-0.2557, -0.2605, -0.4679],
        [-0.2041, -0.2834, -0.5574],
        [-0.2786, -0.2244, -0.3632],
        [-0.2552, -0.2214, -0.3703],
        [-0.3241, -0.2302, -0.3493],
        [-0.2788, -0.2244, -0.3631],
        [-0.3241, -0.2302, -0.3493],
        [-0.3241, -0.2302, -0.3493],
        [-0.3090, -0.2282, -0.3539],
        [-0.1884, -0.2129, -0.3907]], grad_fn=<AddmmBackward0>)

In [38]:
loss = F.cross_entropy(y, t)

In [39]:
loss

tensor(1.0882, grad_fn=<NllLossBackward0>)

In [40]:
# 勾配の算出
loss.backward()

In [41]:
# パラメータの更新
optimizer.step()

In [42]:
# GPUの有無を確認
torch.cuda.is_available()

True

In [45]:
# デバイスの選択
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [46]:
net.to(device)

Net(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
)

In [47]:
x = x.to(device)

In [48]:
t = t.to(device)

In [49]:
# 勾配情報の初期化
optimizer.zero_grad()

In [50]:
## 学習ループ
# エポック数
max_epoch = 1
torch.manual_seed(0)

In [51]:
# モデルのインスタンス化とデバイスへの転送
net = Net().to(device)

In [52]:
# 最適化手法
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

In [53]:
for eposh in range(max_epoch):
  for batch in train_loader:
    x,t = batch
    x = x.to(device)
    t = t.to(device)
    y = net(x)
    loss = F.cross_entropy(y, t)
    print(f'loss: {loss}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

loss: 1.0881630182266235
loss: 1.0393922328948975
loss: 1.002811312675476
loss: 1.0250868797302246
loss: 1.0088638067245483
loss: 0.935197651386261
loss: 0.8939587473869324
loss: 0.9765416979789734
loss: 0.9651519060134888


In [54]:
## 評価指標の追加

In [55]:
x, t = next(iter(train_loader))
x = x.to(device)
t = t.to(device)
y = net(x)
y

tensor([[ 0.2274, -0.4884, -1.0432],
        [-0.2485, -0.2863, -0.4631],
        [ 0.2858, -0.5131, -1.1144],
        [-0.3949, -0.2241, -0.2846],
        [ 0.2245, -0.4871, -1.0397],
        [-0.3949, -0.2241, -0.2846],
        [ 0.2436, -0.4952, -1.0629],
        [-0.3303, -0.2515, -0.3633],
        [-0.3204, -0.2557, -0.3754],
        [ 0.2183, -0.4845, -1.0321]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [56]:
# 結果のラベリング
y_label = torch.argmax(y, dim=1)
y_label

tensor([0, 0, 0, 1, 0, 1, 0, 1, 1, 0], device='cuda:0')

In [57]:
# 正解率の算出
y_label == t

tensor([ True, False,  True, False,  True, False,  True,  True,  True,  True],
       device='cuda:0')

In [58]:
(y_label == t).sum()

tensor(7, device='cuda:0')

In [59]:
(y_label == t).sum().float()

tensor(7., device='cuda:0')

In [61]:
accuracy = (y_label == t).sum().float() / len(t)
accuracy

tensor(0.7000, device='cuda:0')